토픽 모델링은 문서의 집합에서 토픽을 찾아내는 프로세스를 말합니다.  
이는 검색 엔진, 고객 민원 시스템 등과 같이 문서의 주제를 알아내는 일이 중요한 곳에서 사용됩니다.  
잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)은 토픽 모델링의 대표적인 알고리즘입니다. 줄여서 LDA라고 합니다.

LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다.  
데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적합니다.

# 1. 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA) 개요

우선 LDA의 내부 메커니즘에 대해서 이해하기 전에, LDA를 일종의 블랙 박스로 보고 LDA에 문서 집합을 입력하면, 어떤 결과를 보여주는지 간소화 된 예를 들어 보겠습니다.  
아래와 같은 3개의 문서가 있다고 합시다. 지금의 예제는 간단해서 눈으로도 토픽 모델링을 할 수 있을 것 같지만, 실제 수십만개 이상의 문서가 있는 경우는 직접 토픽을 찾아내는 것이 어렵기 때문에 LDA의 도움이 필요합니다.  
  
문서1 : 저는 사과랑 바나나를 먹어요  
문서2 : 우리는 귀여운 강아지가 좋아요  
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요  
  
LDA를 수행할 때 문서 집합에서 토픽이 몇 개가 존재할지 가정하는 것은 사용자가 해야 할 일입니다.  
여기서는 LDA에 2개의 토픽을 찾으라고 요청하겠습니다. 토픽의 개수를 의미하는 변수를 k라고 하였을 때, k를 2로 한다는 의미입니다.  
k의 값을 잘못 선택하면 원치않는 이상한 결과가 나올 수 있습니다. 이렇게 모델의 성능에 영향을 주는 사용자가 직접 선택하는 매개변수를 머신 러닝 용어로 하이퍼파라미터라고 합니다.   
이러한 하이퍼파라미터의 선택은 여러 실험을 통해 얻은 값일 수도 있고, 우선 시도해보는 값일 수도 있습니다.  
  
LDA가 위의 세 문서로부터 2개의 토픽을 찾은 결과는 아래와 같습니다. 여기서는 LDA 입력 전에 주어와 불필요한 조사 등을 제거하는 전처리 과정은 거쳤다고 가정합니다.  '
 즉, 전처리 과정을 거친 DTM이 LDA의 입력이 되었다고 가정합니다.  
  
LDA는 **각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정**합니다.  
  
<각 문서의 토픽 분포>  
문서1 : 토픽 A 100%  
문서2 : 토픽 B 100%  
문서3 : 토픽 B 60%, 토픽 A 40%  
  
<각 토픽의 단어 분포>  
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%  
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%  
  
LDA는 토픽의 제목을 정해주지 않지만, 이 시점에서 알고리즘의 사용자는 위 결과로부터 두 토픽이 각각 과일에 대한 토픽과 강아지에 대한 토픽이라고 판단해볼 수 있습니다.  
이제 LDA에 대해서 알아봅시다. 실제로 LDA는 아래의 설명보다 훨씬 더 복잡하지만, 여기서는 수학적인 수식은 배제하고 개념적 이해에 초점을 둡니다.  

# 2. LDA의 가정

LDA는 문서의 집합으로부터 어떤 토픽이 존재하는지를 알아내기 위한 알고리즘입니다.  
LDA는 앞서 배운 빈도수 기반의 표현 방법인 BoW의 행렬 DTM 또는 TF-IDF 행렬을 입력으로 하는데,   
이로부터 알 수 있는 사실은 LDA는 단어의 순서는 신경쓰지 않겠다는 겁니다.    
    
LDA는 문서들로부터 토픽을 뽑아내기 위해서 이러한 가정을 염두해두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 이러한 생각을 했습니다.  
'나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.' 조금 더 구체적으로 알아보겠습니다.   
각각의 문서는 다음과 같은 과정을 거쳐서 작성되었다고 가정합니다.  
  
**1) 문서에 사용할 단어의 개수 N을 정합니다.**  
 Ex) 5개의 단어를 정하였습니다.  
**2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정합니다.**  
 Ex) 위 예제와 같이 토픽이 2개라고 하였을 때 강아지 토픽을 60%, 과일 토픽을 40%와 같이 선택할 수 있습니다.  
**3) 문서에 사용할 각 단어를 (아래와 같이) 정합니다.**  
**3-1) 토픽 분포에서 토픽 T를 확률적으로 고릅니다.**  
 Ex) 60% 확률로 강아지 토픽을 선택하고, 40% 확률로 과일 토픽을 선택할 수 있습니다.  
**3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고릅니다.**  
 Ex) 강아지 토픽을 선택하였다면, 33% 확률로 강아지란 단어를 선택할 수 있습니다. 이제 3)을 반복하면서 문서를 완성합니다.  
  
이러한 과정을 통해 문서가 작성되었다는 가정 하에 LDA는 토픽을 뽑아내기 위하여 위 과정을 역으로 추적하는 역공학(reverse engneering)을 수행합니다.

# 3. LDA의 수행하기

이제 LDA의 수행 과정을 정리해보겠습니다.

1) 사용자는 알고리즘에게 토픽의 개수 k를 알려줍니다.
앞서 말하였듯이 LDA에게 토픽의 개수를 알려주는 역할은 사용자의 역할입니다. LDA는 토픽의 개수 k를 입력받으면, k개의 토픽이 M개의 전체 문서에 걸쳐 분포되어 있다고 가정합니다.

2) 모든 단어를 k개 중 하나의 토픽에 할당합니다.
이제 LDA는 모든 문서의 모든 단어에 대해서 k개 중 하나의 토픽을 랜덤으로 할당합니다. 이 작업이 끝나면 각 문서는 토픽을 가지며, 토픽은 단어 분포를 가지는 상태입니다. 물론 랜덤으로 할당하였기 때문에 사실 이 결과는 전부 틀린 상태입니다. 만약 한 단어가 한 문서에서 2회 이상 등장하였다면, 각 단어는 서로 다른 토픽에 할당되었을 수도 있습니다.

3) 이제 모든 문서의 모든 단어에 대해서 아래의 사항을 반복 진행합니다. (iterative)

3-1) 어떤 문서의 각 단어 w는 자신은 잘못된 토픽에 할당되어져 있지만, 다른 단어들은 전부 올바른 토픽에 할당되어져 있는 상태라고 가정합니다. 이에 따라 단어 w는 아래의 두 가지 기준에 따라서 토픽이 재할당됩니다.
- p(topic t | document d) : 문서 d의 단어들 중 토픽 t에 해당하는 단어들의 비율
- p(word w | topic t) : 단어 w를 갖고 있는 모든 문서들 중 토픽 t가 할당된 비율

이를 반복하면, 모든 할당이 완료된 수렴 상태가 됩니다. 두 가지 기준이 어떤 의미인지 예를 들어보겠습니다. 설명의 편의를 위해서 두 개의 문서라는 새로운 예를 사용합니다.



위의 그림은 두 개의 문서 doc1과 doc2를 보여줍니다. 여기서는 doc1의 세번째 단어 apple의 토픽을 결정하고자 합니다.



우선 첫번째로 사용하는 기준은 문서 doc1의 단어들이 어떤 토픽에 해당하는지를 봅니다. doc1의 모든 단어들은 토픽 A와 토픽 B에 50 대 50의 비율로 할당되어져 있으므로, 이 기준에 따르면 단어 apple은 토픽 A 또는 토픽 B 둘 중 어디에도 속할 가능성이 있습니다.



두번째 기준은 단어 apple이 전체 문서에서 어떤 토픽에 할당되어져 있는지를 봅니다. 이 기준에 따르면 단어 apple은 토픽 B에 할당될 가능성이 높습니다. 이러한 두 가지 기준을 참고하여 LDA는 doc1의 apple을 어떤 토픽에 할당할지 결정합니다.

# 4. 잠재 디리클레 할당과 잠재 의미 분석의 차이

LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.  
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

# 5. 실습을 통한 이해

# Reference
-https://lettier.com/projects/lda-topic-modeling/